In [1]:
import jieba
import gensim
import pickle
import os
from itertools import chain
from time import strftime, localtime
from NLP_function import *

ModuleNotFoundError: No module named 'NLP_function'

In [53]:
file = 'C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Project_pogress_20180509.xlsx'
sampid = ['A2023', 'A2042','A2108','A2418','A2172','A4305','A2769',
         'A3455','A3450','A3439','A3435','A3444','A4304','A3691','A32940','A2023',
         'A0000','A4537','A4222','A4382','A4442','A3581','A4583','A0000']

In [54]:
# use original clinical data
df = pd.read_excel(file)
df = df.iloc[:, np.r_[0, 5, 17, 21:58]]
df = df.loc[df['样本编号'].isin(sampid)]

# merge title and content
for a in range(0,df.shape[1]-1):
    if not a in np.r_[0,1,2,3,7:19,22:df.shape[1]]:
        i = df.columns[a]
        df[i] = df[i].apply(lambda x : str(x) if pd.isnull(x) else i.replace('精阅体格检查\n/','') + str(x) )
        df[i] = df[i].apply(lambda x : x.replace('伊阅',''))  
        df[i] = df[i].apply(lambda x : x.replace('nan', ''))  
        #df = df[~df[i].isin([''])]        
        
# change number into chinese character 
#col = chain(range(3,8),range(9,16), range(29,35)) # for several panel
col = range(3,40)                                  # for all
for c in col:
    df.iloc[:,c] = df.iloc[:,c].apply(lambda x : change_number(x,'cn'))
    
df = df.drop_duplicates()




# transform into list and remove float phenotype
df = df.T
df.columns = df.iloc[0]
res = []
for i in range(0,df.shape[1]):
    res.append(pd.Series(df.iloc[0:,i]).dropna().drop_duplicates().values) 
for i in range(0,len(res)):
    res[i] = [x for x in res[i] if type(x) != float] 

    
# tokenization
for a in range (0,len(res)):
    for b in range(0,len(res[a])):
        #print(res[a][b])
        res[a][b] = re.split('[，]|[,]|[；]|[：]', res[a][b])
        #print(res[a][b])      

        
# extract information
txt = pd.DataFrame(columns=['id', 'pheno'])

for i in range(0,len(res)):
    length = len(list(chain(*res[i][2:len(res[i])])))
    tem = pd.DataFrame({'id': pd.DataFrame(res[i][0]).values.repeat(length), 'pheno': list(chain(*res[i][2:len(res[i])])) })
    txt = txt.append(tem)
txt['pheno'] = txt['pheno'].dropna()
txt = txt.drop_duplicates()
txt = txt.reset_index(drop=True)


# clean txt (remove non sense doc)
searchfor = ["正常",'零次']

txt['pheno'] = txt['pheno'].apply(lambda x : clean_text(x, lan='cn'))
txt['pheno'] = txt['pheno'].dropna()
txt= txt[txt.pheno!='nan']

txt = txt[~txt.pheno.str.contains('|'.join(searchfor), na=False)] # remove str contraing specific str

txt = txt[~txt.pheno.isin(['', '.', '重点关注','无','既往病史','全外扫描','未查','B超','其他遗传学检查结果','其他检查','要求全外扫描'])] 
txt['pheno'] = txt['pheno'].apply(lambda x: x.replace('45,X','45_X'))

txt = txt.reset_index(drop=True)


# jieba cut
txt['pheno_cut'] = txt['pheno'].apply(lambda txt: list(jieba.cut(txt))) #调用结巴分词

In [56]:
txt.to_csv('clinical_cut_'+ strftime("%Y%m%d_%H%M", localtime()) +'.csv', header=True, index=False, encoding="ANSI")